In [1]:
import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header

import time

In [2]:
with open('./api.txt', 'r') as api_txt:
    api = api_txt.read()

api

'2BeBXg6KgXfwZr8i2P2L7H5W6ujr4XXcy3hTGWmJADw69bQB1Z2J'

In [3]:
directory_in = './file/4.link'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
file = list_file[0]

file

'crawler_1.xlsx'

In [4]:
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                       header=0,
                                                                       dtype=str).fillna('')

df_input['Part Code'] = df_input['Part Code'].astype(int)
df_input['Make Code'] = df_input['Make Code'].astype(int)
df_input['Link No'] = df_input['Link No'].astype(int)

count_remain = len(df_input)

print(f'总数量：{count_remain}')
print()
df_input

总数量：14548



,Part Code,Make,Make Code,Url,JOIN_PCMC,Link No,Json_Link,JOIN_JOIN_PCMCLN
0,13362498,LEXUS,842,https://web.tecalliance.net/tecdoc/qa/parts/21...,13362498;842,1,"{""array"": [{""articleLinkId"": 994698707, ""linki...",13362498;842;1
1,13362498,TOYOTA,111,https://web.tecalliance.net/tecdoc/qa/parts/21...,13362498;111,1,"{""array"": [{""articleLinkId"": 994698706, ""linki...",13362498;111;1
2,13362499,DACIA,139,https://web.tecalliance.net/tecdoc/qa/parts/21...,13362499;139,1,"{""array"": [{""articleLinkId"": 1200379749, ""link...",13362499;139;1
3,13362499,MITSUBISHI,77,https://web.tecalliance.net/tecdoc/qa/parts/21...,13362499;77,1,"{""array"": [{""articleLinkId"": 1430072635, ""link...",13362499;77;1
4,13362499,RENAULT,93,https://web.tecalliance.net/tecdoc/qa/parts/21...,13362499;93,1,"{""array"": [{""articleLinkId"": 815720483, ""linki...",13362499;93;1
...,...,...,...,...,...,...,...,...
14543,1396752376,MERCEDES-BENZ,74,https://web.tecalliance.net/tecdoc/qa/parts/21...,1396752376;74,3,"{""array"": [{""articleLinkId"": 1396752454, ""link...",1396752376;74;3
14544,1396752376,MERCEDES-BENZ,74,https://web.tecalliance.net/tecdoc/qa/parts/21...,1396752376;74,4,"{""array"": [{""articleLinkId"": 1396752407, ""link...",1396752376;74;4
14545,1396752464,MERCEDES-BENZ,74,https://web.tecalliance.net/tecdoc/qa/parts/21...,1396752464;74,1,"{""array"": [{""articleLinkId"": 1396752487, ""link...",1396752464;74;1
14546,1396752464,MERCEDES-BENZ,74,https://web.tecalliance.net/tecdoc/qa/parts/21...,1396752464;74,2,"{""array"": [{""articleLinkId"": 1396752471, ""link...",1396752464;74;2


In [5]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 Part Code                                                    13362498
 Make                                                            LEXUS
 Make Code                                                         842
 Url                 https://web.tecalliance.net/tecdoc/qa/parts/21...
 JOIN_PCMC                                                13362498;842
 Link No                                                             1
 Json_Link           {"array": [{"articleLinkId": 994698707, "linki...
 JOIN_JOIN_PCMCLN                                       13362498;842;1
 Name: 0, dtype: object)

In [6]:
import time

data = {'getArticleLinkedAllLinkingTargetsByIds3': {'articleCountry': 'DE',
                                                    'articleId': crawler_series['Part Code'],
                                                    'immediateAttributs': True,
                                                    'lang': 'qa',
                                                    'linkedArticlePairs': json.loads(crawler_series['Json_Link']),
                                                    'linkingTargetType': 'VOLB',
                                                    'provider': 23365}}

count_retry = 0
while True:
    count_retry += 1
    print(count_retry)
    try:
        url_check = crawler_series['Url']

        resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(**{'x-api-key': api}),
                             timeout=(5, 15))

        if resp.status_code == 200:
            dict_ = resp.json()

            if dict_['status'] == 401 and dict_['statusText'].strip() == 'Access not allowed':
                print(f'fuck api {api}')
                time.sleep(3)
                with open('./api.txt', 'r') as api_txt:
                    api = api_txt.read()
            elif dict_['status'] == 200:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

resp

1


<Response [200]>

In [7]:
list_vehicle = dict_['data']['array']

list_vehicle

[{'articleLinkId': 994698707,
  'linkedArticleImmediateAttributs': {'array': [{'attrBlockNo': 0,
     'attrId': 4224,
     'attrIsConditional': False,
     'attrIsInterval': False,
     'attrName': 'Number of Doors',
     'attrShortName': 'Number of doors',
     'attrType': 'K',
     'attrValue': '5'}]},
  'linkedVehicles': {'array': [{'carDesc': '200h (ZWA10_, ZWA10R)',
     'carId': 53026,
     'constructionType': 'Hatchback',
     'cylinderCapacity': 1798,
     'linkingTargetType': 'P',
     'manuDesc': 'LEXUS',
     'manuId': 842,
     'modelDesc': 'CT (ZWA10_)',
     'modelId': 9014,
     'powerHpFrom': 136,
     'powerHpTo': 136,
     'powerKwFrom': 100,
     'powerKwTo': 100,
     'yearOfConstructionFrom': 201109}]},
  'linkingTargetId': 53026}]

In [8]:
list_vehicle

[{'articleLinkId': 994698707,
  'linkedArticleImmediateAttributs': {'array': [{'attrBlockNo': 0,
     'attrId': 4224,
     'attrIsConditional': False,
     'attrIsInterval': False,
     'attrName': 'Number of Doors',
     'attrShortName': 'Number of doors',
     'attrType': 'K',
     'attrValue': '5'}]},
  'linkedVehicles': {'array': [{'carDesc': '200h (ZWA10_, ZWA10R)',
     'carId': 53026,
     'constructionType': 'Hatchback',
     'cylinderCapacity': 1798,
     'linkingTargetType': 'P',
     'manuDesc': 'LEXUS',
     'manuId': 842,
     'modelDesc': 'CT (ZWA10_)',
     'modelId': 9014,
     'powerHpFrom': 136,
     'powerHpTo': 136,
     'powerKwFrom': 100,
     'powerKwTo': 100,
     'yearOfConstructionFrom': 201109}]},
  'linkingTargetId': 53026}]

In [9]:
list_dict_correct_temp = []
for i, vehicle in enumerate(list_vehicle):
    if len(vehicle['linkedVehicles']['array']) != 1:
        print('fuck')

    if crawler_series['Make'] != vehicle['linkedVehicles']['array'][0]['manuDesc'].strip():
        raise

    dict_vehicle = {'Part Code': crawler_series['Part Code'],
                    'Link No': crawler_series['Link No'],
                    'Json_Link': crawler_series['Json_Link'],
                    'JOIN_PCMCLN': crawler_series['JOIN_JOIN_PCMCLN'],
                    'Vehicle No': i + 1,
                    'Make': crawler_series['Make'],
                    'Model': vehicle['linkedVehicles']['array'][0]['modelDesc'].strip(),
                    'Submodel': vehicle['linkedVehicles']['array'][0]['carDesc'].strip(),
                    'Year': f'''{vehicle['linkedVehicles']['array'][0]['yearOfConstructionFrom']}-{vehicle['linkedVehicles']['array'][0]['yearOfConstructionTo']}''' if 'yearOfConstructionTo' in vehicle['linkedVehicles']['array'][0] and vehicle['linkedVehicles']['array'][0]['yearOfConstructionFrom'] != vehicle['linkedVehicles']['array'][0]['yearOfConstructionTo'] else f'''{vehicle['linkedVehicles']['array'][0]['yearOfConstructionFrom']}-''',
                    'kW': vehicle['linkedVehicles']['array'][0]['powerKwFrom'] if vehicle['linkedVehicles']['array'][0]['powerKwFrom'] == vehicle['linkedVehicles']['array'][0]['powerKwTo'] else f'''{vehicle['linkedVehicles']['array'][0]['powerKwFrom']}-{vehicle['linkedVehicles']['array'][0]['powerKwTo']}''',
                    'HP': vehicle['linkedVehicles']['array'][0]['powerHpFrom'] if vehicle['linkedVehicles']['array'][0]['powerHpFrom'] == vehicle['linkedVehicles']['array'][0]['powerHpTo'] else f'''{vehicle['linkedVehicles']['array'][0]['powerHpFrom']}-{vehicle['linkedVehicles']['array'][0]['powerHpTo']}''',
                    'cc': vehicle['linkedVehicles']['array'][0]['cylinderCapacity'] if 'cylinderCapacity' in vehicle['linkedVehicles']['array'][0] else '',
                    'Design': vehicle['linkedVehicles']['array'][0]['constructionType'].strip()}

    if type(vehicle['linkedArticleImmediateAttributs']) == str and vehicle['linkedArticleImmediateAttributs'].strip() == '':
        list_attribute = []
    elif type(vehicle['linkedArticleImmediateAttributs']) == dict:
        list_attribute = vehicle['linkedArticleImmediateAttributs']['array']
    else:
        print('fuck')

    dict_count = {}
    for attribute in list_attribute:
        if 'attrValue' not in attribute:
            continue
        key = attribute['attrName'].strip()
        if key in dict_count:
            dict_count[key] += 1
        else:
            dict_count[key] = 1

    dict_no = {}
    for attribute in list_attribute:
        if attribute['attrName'].strip() == 'Block Separation' and attribute['attrShortName'].strip() == 'or' and 'attrValue' not in attribute:
            continue
        key = attribute['attrName'].strip()
        if dict_count[key] == 1:
            dict_vehicle[key] = attribute['attrValue'].strip()
        else:
            if key in dict_no:
                dict_no[key] += 1
            else:
                dict_no[key] = 1
            dict_vehicle[f'{key}-{dict_no[key]}'] = attribute['attrValue'].strip()

    list_dict_correct_temp.append(dict_vehicle)

list_dict_correct_temp

[{'Part Code': 13362498,
  'Link No': 1,
  'Json_Link': '{"array": [{"articleLinkId": 994698707, "linkingTargetId": 53026}]}',
  'JOIN_JOIN_PCMCLN': '13362498;842;1',
  'Vehicle No': 1,
  'Make': 'LEXUS',
  'Model': 'CT (ZWA10_)',
  'Submodel': '200h (ZWA10_, ZWA10R)',
  'Year': '201109-',
  'kW': 100,
  'HP': 136,
  'cc': 1798,
  'Design': 'Hatchback',
  'Number of Doors': '5'}]

In [10]:
list_dict_correct.extend(list_dict_correct_temp)

list_dict_correct

[{'Part Code': 13362498,
  'Link No': 1,
  'Json_Link': '{"array": [{"articleLinkId": 994698707, "linkingTargetId": 53026}]}',
  'JOIN_JOIN_PCMCLN': '13362498;842;1',
  'Vehicle No': 1,
  'Make': 'LEXUS',
  'Model': 'CT (ZWA10_)',
  'Submodel': '200h (ZWA10_, ZWA10R)',
  'Year': '201109-',
  'kW': 100,
  'HP': 136,
  'cc': 1798,
  'Design': 'Hatchback',
  'Number of Doors': '5'}]

In [11]:
crawler_status = True

crawler_status

True

In [12]:
crawler_series['Check_Url'] = url_check

list_series_error.append(crawler_series)

list_series_error

[Part Code                                                    13362498
 Make                                                            LEXUS
 Make Code                                                         842
 Url                 https://web.tecalliance.net/tecdoc/qa/parts/21...
 JOIN_PCMC                                                13362498;842
 Link No                                                             1
 Json_Link           {"array": [{"articleLinkId": 994698707, "linki...
 JOIN_JOIN_PCMCLN                                       13362498;842;1
 Check_Url           https://web.tecalliance.net/tecdoc/qa/parts/21...
 Name: 0, dtype: object]

In [13]:
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Part Code', 'Make', 'Link No', 'Vehicle No'],
                                                             ascending=[True, True, True, True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_vehicle.xlsx', index=False)

df_correct

,Part Code,Link No,Json_Link,JOIN_JOIN_PCMCLN,Vehicle No,Make,Model,Submodel,Year,kW,HP,cc,Design,Number of Doors
0,13362498,1,"{""array"": [{""articleLinkId"": 994698707, ""linki...",13362498;842;1,1,LEXUS,CT (ZWA10_),"200h (ZWA10_, ZWA10R)",201109-,100,136,1798,Hatchback,5


In [14]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['Part Code', 'Make', 'Link No'],
                                                           ascending=[True, True, True],
                                                           ignore_index=True)
    df_error.to_excel('./test_vehicle_error.xlsx', index=False)

df_error

,Part Code,Make,Make Code,Url,JOIN_PCMC,Link No,Json_Link,JOIN_JOIN_PCMCLN,Check_Url
0,13362498,LEXUS,842,https://web.tecalliance.net/tecdoc/qa/parts/21...,13362498;842,1,"{""array"": [{""articleLinkId"": 994698707, ""linki...",13362498;842;1,https://web.tecalliance.net/tecdoc/qa/parts/21...
